In [4]:
import pandas as pd
import numpy as np

In [5]:
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [6]:
data = pd.read_csv('transaction_data.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [7]:
item_features = pd.read_csv('product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [8]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65..."
1,3,"[823704, 834117, 840244, 913785, 917816, 93870..."


In [9]:
MyRecommeder = MainRecommender(data_train)

C:\Users\SV\anaconda3\envs\recsys\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

In [10]:
MyRecommeder.get_similar_items_recommendation(95)

[934639, 879755, 899844, 1068292, 1113111]

In [11]:
MyRecommeder.get_similar_items_recommendation(1)

[985999, 1133018, 1137346, 5569845, 1106523]

In [12]:
MyRecommeder.get_similar_users_recommendation(5)

[1126899, 921345, 1082185, 1082185, 1075368]

In [24]:
bought_list = result[result['user_id'] == 1]['actual'][0]
bought_list

array([  879517,   934369,  1115576,  1124029,  5572301,  6514011,
       13115375, 13876665, 15926844,   877373,   877391,   936685,
         938004,   940947,   953561,   959219,   971585,   976767,
         977497,   986947,  1026056,  1035805,  1036852,  1043064,
        1049998,  1062002,  1066509,  1074612,  1082185,  1094919,
        1109465,  1132771,  8293382,  8293898,  9527558, 12987976,
       13115808, 15800711, 15926712, 15973164, 17901109,   837569,
         849250,   973135,  1070238,  1087986, 15597040,  1137100,
         840361,   891354,   924611,   936110,   951907,   958046,
         970619,   991580,   995242,  1050229,  1065167,  1069575,
        1082212,  1103741,  8293439,  9677939,  9834742, 10149640,
       13382402, 17179899, 17330026,   820165,   830304,   835796,
         865900,   898011,   908514,   949311,   961554,   977658,
         979707,   988736,  1004906,  1005186,  1041796,  1055646,
        1065910,  5585510,  5978648,  8090541,  8090993, 12352

In [25]:
recommended_list = MyRecommeder.get_similar_items_recommendation(1)
recommended_list

[985999, 1133018, 1137346, 5569845, 1106523]

In [26]:
precision_at_k(recommended_list, bought_list)

0.0

In [27]:
#заменим один товар, чтобы проверить работу функции - всё корректно
precision_at_k([879517, 1133018, 1137346, 5569845, 1106523], bought_list)

0.2